In [ ]:
import os
import nltk
from pymorphy2 import MorphAnalyzer
pmm = MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
from pymorphy2.tokenizers import simple_word_tokenize
tokenizer = RegexpTokenizer(r'\w+')

import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from nltk.parse import DependencyGraph
from collections import Counter
from nltk.collocations import *
import scipy
from scipy import stats

def normalize_text(text):
    text = tokenizer.tokenize(text.lower())
    lemmas = [pmm.parse(t)[0].normal_form for t in text]
    return ' '.join(lemmas)

#### Парсим корпус

In [ ]:
# !C:\Users\qwe\Desktop\cosyco\udpipe\udpipe-1.2.0-bin\bin-win64\udpipe --input horizontal --output conllu \
# --tokenize --tag --parse \
# C:\Users\qwe\Desktop\cosyco\udpipe\russian-syntagrus-ud-2.4-190531.udpipe < testset2.txt > text.conllu

In [ ]:
trees = []
with open('text.conllu', 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')
    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))
        

print(trees[2])

#### Строим частотный словарь глаголов

In [ ]:
verb_freq = Counter()
for one_tree in trees:
    try:
        g = DependencyGraph(one_tree, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['ctag'] == 'VERB':
                verb_freq[g.nodes[n]['lemma']] += 1
    except:
        pass
verb_freq_50 = [item[0] for item in verb_freq.most_common() if item[1] >= 50]
print(verb_freq_50)

#### Собираем все биграммы (глагол + прямое дополнение)

-> биграммы в виде лемм

In [ ]:
#bigrams = Counter()
bigrams = []
for one_tree in trees:
    try:
        g = DependencyGraph(one_tree, top_relation_label='root')
        for item in g.triples():
            if item[1] == 'obj' and item[2][1] == 'NOUN':
                lemma = normalize_text(item[0][0])
                if lemma in verb_freq_50:
                    coll = tuple([normalize_text(item[0][0]), 
                                  normalize_text(item[2][0])])
                    bigrams.append(coll)
                    # bigrams[tuple([item[0][0], item[2][0]])] += 1
    except:
        pass

In [ ]:
bigrams[:20]

#### Оцениваем по метрикам log-likelihood, dice, PMI

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(bigrams)
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in nltk.corpus.stopwords.words('russian'))

In [ ]:
loglike_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.likelihood_ratio)}
pmi_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.pmi)}
dice_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.dice)}

In [ ]:
pmi_100 = finder.nbest(bigram_measures.pmi, 100)
loglike_100 = finder.nbest(bigram_measures.likelihood_ratio, 100)
dice_100 =  finder.nbest(bigram_measures.dice, 100)

Находим пересечение трех метрик

In [ ]:
three_metrics = set(pmi_100) & set(loglike_100) & set(dice_100)
three_metrics

Словарь глагольной сочетаемости

* извлекае все словосочетания (из последней колонки) в том порядке, в котором они даны
* лемматизируем



In [ ]:
with open('verb_coll.txt', 'r', encoding='utf-8') as verb_coll:
    verb_colls = verb_coll.read().split('\n')

verb_colls = [t.split('\t')[-1] for t in verb_colls]
verb_colls = [tuple(normalize_text(t).split()) for t in verb_colls]

#### Находим Золотой стандарт

In [ ]:
Gold = set(verb_colls) & set(pmi_100) & set(loglike_100) & set(dice_100)
Gold

In [ ]:
# Что осталось за пределами ЗС?
three_metrics - Gold

К ЗС можно было бы добавить следующие коллокации:

* Объявить голодовку
* заявить отвод
* взыскать неустойку

Критерием выделения коллокаций можно считать их фразеологичность и ограниченную сочетаемость составляющих коллокацию слов. Т.е чтобы проверить, является ли словосочетание коллокацией, нужно попробовать заменить слова

Попробуем: 

* *сказать/проговорить/пообещать/заявить/прокричать голодовку
* *объявить/сказать/проговорить отвод
* ?собрать/взять/забрать неустойку (сказать что-то типа потребовать неустойку, наверное, можно, но взыскать неустойку звучит привычнее)


In [ ]:
Gold |= set([('взыскать', 'неустойка'), ('заявить', 'отвод'), ('объявить', 'голодовка')])
Gold

#### Считаем ранговую корреляцию

In [ ]:
data_loglike = dict.fromkeys(Gold)
for item in Gold:
    data_loglike[item] = loglike_scores[item]
data_loglike

In [ ]:
data_pmi = dict.fromkeys(Gold)
for item in Gold:
    data_pmi[item] = pmi_scores[item]
data_pmi

In [ ]:
data_dice = dict.fromkeys(Gold)
for item in Gold:
    data_dice[item] = dice_scores[item]
data_dice

* Если считать, что коллокации в золотом стандарте - эталон, и приписать им всем оценку, близкую к 1.0, то корреляции не будет
* Поэтому для каждой коллокации я попробовала взять ее усредненную оценку для всех трех метрик

In [ ]:
data0 = [0.9] * 7
stats_m = [
    list(data_loglike.values()),
    list(data_pmi.values()),
    list(data_dice.values())
]

In [ ]:
for i in stats_m:
    print(stats.spearmanr(data0, i))

*берем среднее между тремя метриками*

In [ ]:
data1 = dict.fromkeys(Gold)
for colloc in data1:
    data1[colloc] = np.mean([pmi_scores[colloc], loglike_scores[colloc], dice_scores[colloc]])
data1

In [ ]:
for i in stats_m:
    print(stats.spearmanr(list(data1.values()), i))

Попробуем сами отранжировать коллокации

In [ ]:
data3 = [6, 3, 2, 4, 7, 5, 1]
for i in stats_m:
    print(stats.spearmanr(list(data3), i))

Наибольшее значение корреляции у pmi, у других метрик значение корреляции сильно хуже

PMI принимает во внимание именно то, насколько часто слова встречаются вместе, при учете их вероятностей по-отдельности

Возможно, коллокации - это редкие слова по-отдельности, которые встречаются зачастую вместе, поэтому по-отдельности они не так значимы, как их сочетание

Попробуем посчитать рандомную корреляцию

In [ ]:
np.random.seed(42)
data2 =  np.random.random_sample((7,))
for i in stats_m:
    print(stats.spearmanr(data2, i))
data2

In [ ]:
set(pmi_100[:30]) & Gold, set(pmi_100[:50]) & Gold

In [ ]:
set(loglike_100[:30]) & Gold, set(loglike_100[:50]) & Gold

In [ ]:
set(dice_100[:30]) & Gold, set(dice_100[:50]) & Gold

In [ ]:
loglike_scores

In [ ]:
pmi_scores

In [ ]:
dice_scores

### Анализ

* Оценка корреляции показала, что ближе всего к ЗС оказалась метрика loglikelihood
* Меньше всего корреляция для PMI, на втором месте расположилась dice
* Если мы сами оценим, насколько высокую оценку дают метрики для коллокаций, вошедших в ЗС, то увидим, что точнее всего оказывается dice: в первых 30 коллокаций встретилось больше сочетаний,вошедших в ЗС, чем для остальных метрик
* Кажется, что для pmi важна еще и частотность отдельного слова в коллокации, если посмотреть на выдачу, то можно заметить, будто распределение коллокаций как будто сгруппировано по отдельным словам - вершинам коллокаций
* Loglikelihood скорее в данном случае выбрал самые частотные сочетания слов, самые частотные биграммы. Вопрос: частотное словосочетание == коллокация?
* Dice учитывает совместные сочетания слов, и чем частотнее слова встречаются вместе, чем меньше частотность отдельных слов, тем лучше. Однако ошибочны просто редкие сочетания, когда коллокация встретилась дишь один раз, и слова в ней встретились только в этой коллокации, оказываются в топе, что приводит к ошибочному выделению коллокации (('рассказать', 'замгендиректор'), ('являться', 'долг'))